In [1]:
import numpy as np
import pandas as pd

## Birbirine bağlama
Her iki kaynak aynı formatta ise **pd.concat()** yöntemini kullanmaktır

*concatenation** yöntemi basitce sütunlarla veya satırlarla iki DataFrame'i yapıştırır

**Aynı formatta olan iki DataFrame'i birbirine yapıştırma**

In [2]:
data_one = {'A': ['A0', 'A1', 'A2', 'A3'],'B': ['B0', 'B1', 'B2', 'B3']}

In [3]:
data_two = {'C': ['C0', 'C1', 'C2', 'C3'], 'D': ['D0', 'D1', 'D2', 'D3']}

In [4]:
one = pd.DataFrame(data_one)

In [5]:
two = pd.DataFrame(data_two)

In [6]:
one

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [7]:
two

,C,D
0,C0,D0
1,C1,D1
2,C2,D2
3,C3,D3


### Satır boyunca birleştirme (axis = 0) 

In [8]:
axis0 = pd.concat([one,two],axis=0)

In [9]:
axis0

,A,B,C,D
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
2,A2,B2,NaN,NaN
3,A3,B3,NaN,NaN
0,NaN,NaN,C0,D0
1,NaN,NaN,C1,D1
2,NaN,NaN,C2,D2
3,NaN,NaN,C3,D3


### Axis=0, ama sütunlar eşleşiyor
**Sütunların eşleşmesini istemeniz durumunda:**

In [10]:
two.columns = one.columns

In [11]:
mydf = pd.concat([one,two],axis=0)
mydf

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3
0,C0,D0
1,C1,D1
2,C2,D2
3,C3,D3


In [12]:
mydf.index = range(len(mydf))

In [13]:
mydf

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3
4,C0,D0
5,C1,D1
6,C2,D2
7,C3,D3


Peki bu neden böyle oldu?

Bunun ana nedeni, C ve D sütunları ile A ve B sütunları arasındaki farkı belirtmektir, çünkü aynı konumda iki değerin dört indeksi sıfır olamaz. Dolayısıyla, sıfır indeks için A ve C arasında seçim yapmak yerine, yaptığı şey esasen 0, 1, 2, 3 endeksini çoğaltıyor ve daha sonra 0, 1, 2, 3 indeksini tekrarlıyor. Bu nedenle, A ve B'deki sıfır indeksi için, orijinal veri indeksinde, C ve D'nin herhangi bir değeri yoktur. Size bu two DataFrame'ine eklemenin iyi bir fikir olmadığını bildirmek, aynı indeksin çağrılarını paylaştıkları için gerçekten mantıklı olmadığı için sıfıra eşit değildir.

Burada görebileceğimiz gibi, muhtemelen en mantıklı olan sütunları eklemektir. Ancak, satırlarla eklemek istediğiniz durumu hayal edelim, sadece C ve A ve B'yi yeniden etiketlemek istiyorsunuz, çünkü belki farklı bir veri kümesinde farklı sütun adları vardı.

Bunun pek mantıklı olmadığı görülüyor mantıklı olan sütunları birleştirmek olacaktır.

### Sütun boyunca birleştirme (axis = 1)

In [14]:
axis1 = pd.concat([one,two],axis=1)

In [15]:
axis1

,A,B,A,B
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


# İç Birleştirme (Inner Merge)
Bir veri çerçevesinde bulunan fakat diğerinde bulunmayan satırlar veya sütunlar olduğu durumda veri çerçevelerini nasıl birleştireceğiz.

## Veri Tabloları
Diyelim ki bir şirketimiz var ve film kiralama sektöründeki insanlar için bir tür konferans düzenliyorlar. Şirketteki kişilerin önce online kayıt yaptırmaları gerekiyor. Bilgisayara gidip online kayıt yaptırıyorlar ve kimliklerini, yani hangi sırayla kayıt yaptırdıklarını ve kişinin adını kaydeden küçük bir takipçimiz var. Ve daha sonra, konferansa gerçekten geldiklerinde, yapacakları şey fiziksel konferans sitesindeki isimlerini girmeleri ve ardından giriş kimliklerini söylemeleridir, esasen, fiziksel konferansa hangi sırayla ve isimle geldiler?

Konferanstan sonra iki veri tablomuz var. Birinin kayıt tablomuz olduğuna dikkat edin, kayıt kimliğimiz var, insanların hangi sırayla kaydolduğu ve ardından bu kişilerin adları: Andrew, Bob, Charlie, David ve sonra bir oturum açma tablomuz var, fiziksel sitede gerçekten oturum açan kişiler.

Burada dikkat edilmesi gereken şey, kayıt yaptıran herkesin konferansa gidip giriş yapmamış olmasıdır. Belki bir şey oldu ve gelemediler. Ayrıca, fiziksel sitede oturum açan ancak önceden kaydolmayı unutan bazı kişilerin olduğunu da farketmelisiniz.

Bu yüzden yapmamız gereken şey, ilgili ID sütunlarının siteye hangi sırayla kayıt olduklarını veya giriş yaptıklarını gösterdiğini görmek. 

Andrew kayıt yaptıran ilk kişi ardından Xavier fiziksel siteye giriş yapan ilk kişidir. Burada basitlik adına, isimlerin benzersiz olduğunu varsayacağız. Yani, tüm şirkette Andrew, Bob ya da Charlie adında yalnızca bir kişi var.

In [16]:
# Kayıt Tablomuz
registrations = pd.DataFrame({'reg_id':[1,2,3,4],'name':['Andrew','Bobo','Claire','David']})

# Giriş Tablomuz
logins = pd.DataFrame({'log_id':[1,2,3,4],'name':['Xavier','Andrew','Yolanda','Bobo']})

In [17]:
registrations.style.set_caption('Kayıt Tablomuz')

,reg_id,name
0,1,Andrew
1,2,Bobo
2,3,Claire
3,4,David


In [18]:
logins.style.set_caption('Giriş Tablomuz')

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


Pandas'ta birleştirme yöntemi çağrısı gerçekleştirirken, öncelikle hangi sütunu birleştireceğinize karar vermemiz gerekir. Yani bu örnekte dört farklı sütun: kayıt kimliği(reg_id), ad(name), oturum açma kimliği(log_id) ve oturum açma adı(name). 
Bu tabloları birleştirmek için uygun adayın ne olduğuna karar vermeliyiz.

Burada temelde iki kural var. İlki en önemli olanıdır.

**on** sütunu birincil tanımlayıcı olmalıdır, yani aslında o satır için benzersiz bir tanımlayıcı olacaktır.

Burada adların benzersiz olduğunu varsaydığımız için, tablolardan birinde birinin adının Andrew olduğunu biliyorsam, bunun tüm tablo için benzersiz olduğunu bilirim.

Bir **on** sütunu için sahip olmamız gereken diğer durum, birleştirilen her iki tabloda da bulunması ve her iki tabloda da aynı şeyi temsil etmesi gerektiğidir.

Yani bu durumda, **name** her iki tabloda da mevcut ve her iki tabloda da aynı şeyi temsil etmektedir. Sadece kayıt yaptıran ya da giriş yapan kişinin adını temsil eder.

Kayıt kimliği ve oturum açma kimliği sütununa daha derinlemesine bakarsak, aslında aynı şeyi temsil etmediklerini görürüz. Biri insanların kaydolduğu sırayı, diğeri ise giriş yaptığı sırayı temsil eder.

Bu yüzden tek sütunu aklınızda tutun. Her iki tabloda farklı şekilde adlandırılsa da, aslında aynı şeyi temsil ediyor olması gerekir.

Sonra yapacağımız şey isim sütununda birleştirme yapmak olacak. Bu da aslında pd'yi çağırdığımızda, **on** 'un **name** 'e eşit olduğunu söyleyeceğimiz anlamına gelir.

Tabloları name sütununda nasıl birleştireceğimize karar vermemiz gerekiyor. İlk olarak **how = inner** ile başlayacağız, böylece sonuç her iki tabloda da eşleşen kayıtlar kümesi olacaktır.

# pd.merge() Metodu
 - **how** etiketli anahtarın parametreleri:
   * Inner
   * Outer
   * Left ve right

SQL birleştirmeye benzer şekilde, anahtar sütunlara dayalı pandas DataFrame'leri birleştirme. 

Metodun arkasındaki ana fikir, birleştirilen tablolardan yalnızca birinde bulunan bilgilerle nasıl başa çıkılacağına karar vermektir. Yani bir veri çerçevenizi başka bir veri çerçevesiyle birleştiriyorsanız, ne yapacağınıza karar vermeniz gerekir.

Birleştirilmiş sonuçlarda veri çerçevelerinden yalnızca birinde satır varsa, bu sonuçların boş olması veya eksik bilgi olması dışında kalmasını mı yoksa tamamen yok sayılmasını mı istiyorsunuz? İşte bu noktada **how** parametresi devreye girer.

In [19]:
#help(pd.me)

-----

# İç(inner), Sol(left), Sağ(right) ve Dış(outer) Birleşimler

## İç birleşim (inner merge)

**BOTH tablolarında anahtarın bulunduğu yeri eşleştirin. Birleştirme nedeniyle NaN olmamalıdır, çünkü tanım gereği İç birleştirmenin bir parçası olmak için her iki tabloda da bilgiye ihtiyaçları vardır.**
**Sadece Andrew ve Bobo hem kaydoldu hem de giriş yaptı.**

In [20]:
# Notice pd.merge doesn't take in a list like concat
pd.merge(registrations,logins,how='inner',on='name')

,reg_id,name,log_id
0,1,Andrew,2
1,2,Bobo,4


In [21]:
# Pandas, Anahtar sütunu anlayacak kadar akıllı (parametre ile) 
# yalnızca bir sütun adı eşleşirse
pd.merge(registrations,logins,how='inner')

,reg_id,name,log_id
0,1,Andrew,2
1,2,Bobo,4


In [22]:
pd.merge(logins,registrations,how='inner')

,log_id,name,reg_id
0,2,Andrew,1
1,4,Bobo,2


In [23]:
# Pandas, 'on' anahtar sütunu her iki veri çerçevesinde de yoksa hata bildirir
# pd.merge(registrations,logins,how='inner',on='reg_id')

## Sol Birleşim

**Eşleştir ve Sol Tablodaki tüm satırları dahil et.**
**Giriş bilgileri yoksa Sol Tabloya kaydolan herkesi gösterir ve ardından NaN ile doldurur.**

In [25]:
pd.merge(left=registrations,right=logins,how='left',on='name')

,reg_id,name,log_id
0,1,Andrew,2.0
1,2,Bobo,4.0
2,3,Claire,NaN
3,4,David,NaN


## Sağ Birleşim
**Eşleştir ve Sağ Tablodaki tüm satırları dahil et.**
**Sağ Tabloya giriş yapan herkesi göster, kayıt bilgileri yoksa NaN ile doldurur.**

In [26]:
pd.merge(registrations,logins,how='right')

,reg_id,name,log_id
0,NaN,Xavier,1
1,1.0,Andrew,2
2,NaN,Yolanda,3
3,2.0,Bobo,4


## Dış Birleşim

**Sol veya Sağ Tabloda bulunan tüm bilgileri eşleştirin.**
**Registrations ve Logins tablosundaki herkesi gösterir. Eksik bilgileri NaN ile doldurur**

In [27]:
pd.merge(registrations,logins,how='outer')

,reg_id,name,log_id
0,1.0,Andrew,2.0
1,2.0,Bobo,4.0
2,3.0,Claire,NaN
3,4.0,David,NaN
4,NaN,Xavier,1.0
5,NaN,Yolanda,3.0


## İndeks veya Sütun üzerinde Birleştirme

**Bir sütunu veya dizini birbiri üzerinde birleştirmek için left_on, right_on, left_index, right_index kombinasyonlarını kullanın**

In [28]:
registrations

,reg_id,name
0,1,Andrew
1,2,Bobo
2,3,Claire
3,4,David


In [29]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [30]:
# registrations tablosunun indeks adını "name" sütunu olarak ayarlama
registrations = registrations.set_index("name")

In [31]:
registrations

,reg_id
name,
Andrew,1
Bobo,2
Claire,3
David,4


Şimdi indeks olarak bir sütun olan kayıt kimliğine sahibiz. registrations tablosunun indeksi ve logins tablosunun "name" sütununda birleştirme yapmak istersek.

In [32]:
pd.merge(registrations,logins,left_index=True,right_on='name')

,reg_id,log_id,name
1,1,2,Andrew
3,2,4,Bobo


In [33]:
pd.merge(logins,registrations,right_index=True,left_on='name')

,log_id,name,reg_id
1,2,Andrew,1
3,4,Bobo,2


### Birleştirilmiş tablolarda farklı anahtar sütun adları 

In [34]:
registrations = registrations.reset_index()

In [35]:
registrations

,name,reg_id
0,Andrew,1
1,Bobo,2
2,Claire,3
3,David,4


In [36]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [37]:
registrations.columns = ['reg_name','reg_id']

In [38]:
registrations

,reg_name,reg_id
0,Andrew,1
1,Bobo,2
2,Claire,3
3,David,4


In [39]:
# ERROR
# pd.merge(registrations,logins)

In [40]:
pd.merge(registrations,logins,left_on='reg_name',right_on='name')

,reg_name,reg_id,log_id,name
0,Andrew,1,2,Andrew
1,Bobo,2,4,Bobo


In [41]:
pd.merge(registrations,logins,left_on='reg_name',right_on='name').drop('reg_name',axis=1)

,reg_id,log_id,name
0,1,2,Andrew
1,2,4,Bobo


### Pandas yinelenen sütunları otomatik olarak etiketler<

In [42]:
# registration tablosunun sütun isimlerini yeniden tanımlıyoruz.
registrations.columns = ['name','id']

In [43]:
# logins tablosunun sütun isimlerini yeniden tanımlıyoruz.
logins.columns = ['id','name']

In [44]:
registrations

,name,id
0,Andrew,1
1,Bobo,2
2,Claire,3
3,David,4


In [45]:
logins

,id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [46]:
# _x is for left
# _y is for right
pd.merge(registrations,logins,on='name')

,name,id_x,id_y
0,Andrew,1,2
1,Bobo,2,4


In [47]:
# _x ve _y soneklerininin ismini değiştirme
pd.merge(registrations,logins,on='name',suffixes=('_reg','_log'))

,name,id_reg,id_log
0,Andrew,1,2
1,Bobo,2,4


# SON!